In [1]:
import tokenize, ast
from io import BytesIO

In [3]:
!pip install tiktoken
from tiktoken import encoding_for_model
enc = encoding_for_model("text-davinci-003")
toks = enc.encode("They are splashing")
toks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:00


[2990, 389, 4328, 2140]

In [4]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' spl', 'ashing']

# The OpenAI API

In [1]:
from openai import ChatCompletion,Completion

In [2]:
!pip install openai==0.28

In [20]:
import openai
aussie_sys = "You are an American LLM that uses American slang and analogies whenever possible."
# openai.api_key = openai.api_key = "my dummy api key"

c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "who are you?"}])

In [21]:
c['choices'][0]['message']['content']

"Hey there! I'm an American LLM - basically a legal AI assistant here to help you out with any law-related questions you have. Just think of me as your trusty legal sidekick!"

In [22]:
from fastcore.utils import nested_idx

In [23]:
def response(compl): print(nested_idx(compl, 'choices', 0, 'message', 'content'))

In [24]:
response(c)

Hey there! I'm an American LLM - basically a legal AI assistant here to help you out with any law-related questions you have. Just think of me as your trusty legal sidekick!


In [25]:
print(c.usage)

{
  "prompt_tokens": 31,
  "completion_tokens": 40,
  "total_tokens": 71,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}


In [27]:
0.03 / 1000 * 150 # GPT 4
0.002 / 1000 * 150 # GPT 3.5

0.0003

In [28]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [29]:
response(c)

Yeah, mate. Just like how kangaroos help baby kangaroos stay safe in their pouch, money helps you buy stuff and stay safe by providing for your needs. Just as kangaroos hop around, money moves from one person to another through transactions. And just like how a kangaroo's pouch can hold a lot of things, money allows you to save and accumulate wealth.


In [30]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [31]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

Ah, the age-old question! The meaning of life is kind of like a mystery wrapped in an enigma - everyone's got their own take on it. Some folks think it's all about finding happiness and fulfillment, while others believe it's about making a difference in the world. Ultimately, it's up to each individual to figure out what gives their life meaning and purpose. It's a bit like a Choose Your Own Adventure book - you gotta decide what path makes sense for you.


In [32]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return ChatCompletion.create(model=model, messages=msgs)
    except openai.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [34]:
call_api("What's the world's smartest people? Has there ever been any scientific analysis?")

<OpenAIObject chat.completion id=chatcmpl-ARlbqX4iKirgwaDZJKiJyBQaLzz0a at 0x797eb9662110> JSON: {
  "id": "chatcmpl-ARlbqX4iKirgwaDZJKiJyBQaLzz0a",
  "object": "chat.completion",
  "created": 1731181230,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "There is no definitive answer to who the world's smartest people are, as intelligence is a complex and multifaceted trait that is difficult to quantify. There have been various attempts to measure intelligence, such as IQ tests or standardized academic tests, but these have limitations and do not capture the full range of intellectual abilities.\n\nSome well-known figures who have been considered among the smartest people in history include Albert Einstein, Stephen Hawking, and Leonardo da Vinci. However, it is important to note that intelligence is not just about academic or scientific achievements, but also encompasses creativity, problem-solvin

# Create our own code interpreter

In [38]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [39]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [40]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [41]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [42]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [43]:
m = c.choices[0].message
m

<OpenAIObject at 0x797eb9662020> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\"a\":6,\"b\":3}"
  },
  "refusal": null
}

In [44]:
k = m.function_call.arguments
print(k)

{"a":6,"b":3}


In [45]:
funcs_ok = {'sums', 'python'}

In [46]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [47]:
call_func(c)

9

In [50]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [52]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [54]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [57]:
call_func(c)

Proceed with execution?
```
import math
result = math.factorial(12)
result
```
479001600


'#FAIL#'

In [58]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[schema(python)],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "python", "content": "479001600"}])

In [59]:
response(c)

The factorial of 12 is 479,001,600.


In [63]:
c = askgpt("What is the capital of Japan?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [64]:
response(c)

The capital of Japan is Tokyo.


# PyTorch and Huggingface

In [2]:
!huggingface-cli login
import torch


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [4]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
mn = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(mn, load_in_8bit=True, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Cai xu kun, the chinese idol, is"
toks = tokr(prompt, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [17]:
toks

{'input_ids': tensor([[    1,   315,  1794,   921, 29884, 15465, 29892,   278,   521,  8233,
          1178,   324, 29892,   338]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
tokr.batch_decode(toks['input_ids'])

['<s> Cai xu kun, the chinese idol, is']

In [19]:
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
# import torch
# torch.cuda.empty_cache()

In [19]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=30).to('cpu')
res

CPU times: user 7.83 s, sys: 2.11 s, total: 9.93 s
Wall time: 10 s


tensor([[    1,   315,  1794,   921, 29884, 15465, 29892,   278,   521,  8233,
          1178,   324, 29892,   338,   263,  5972, 15640,   322, 11339,   515,
           521,  1099, 29889,   540,   756,  5492, 12727,  7124, 12516,   322,
          5810,  1127,   297,  5164, 11456,  8541,   294,   322,  2351,   583,
         29889,   670, 24909,   526]])

In [20]:
tokr.batch_decode(res)

['<s> Cai xu kun, the chinese idol, is a popular singer and actor from china. he has released numerous hit songs and starred in various television dramas and movies. his fans are']

In [21]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [22]:
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
torch.cuda.empty_cache()
gen(prompt, 50)

['<s> Cai xu kun, the chinese idol, is famous for helping his idol, Justin Bieber. He was one of the most famous Chinese singers, and he gained a lot of popularity in a short time. Cai Xu Kun, or "KUN", has released several']

In [14]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [23]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [24]:
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [25]:
ques = "Who is Cai xu kun?"
gen(mk_prompt(ques), 150)

["<s> ### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n### User: Who is Cai xu kun?\n\n### Assistant:\nCai Xu Kun (蔡徐坤) is a Taiwanese singer, songwriter, and actor. In terms of career, he's been involved in various areas of the entertainment industry, with an emphasis on music and acting. His roles have included lead parts in TV dramas and movies. His musical collaborations have led him to work with other renowned singers and musicians in the region. Known for his strong vocal abilities and his powerful stage presence, he continues to be a prominent figure within the entertainment circuit in Taiwan, Hong Kong, and China.\n\nPlease note this information might be outdated, as it is based on available data at the time this response"]

In [5]:
!pip install optimum
!pip install auto-gptq
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/OpenOrca-Platypus2-13B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_att

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [26]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [27]:
gen(mk_oo_prompt(ques), 150)

["<s> ### Instruction: Who is Cai xu kun?\n\n### Response:\nCai Xu Kun is a contemporary Chinese artist known for his unique artistic style and skillful use of various media. His works often explore human behavior, social issues, and dreamlike sceneries. He has been exhibited at galleries and museums around the world and garnered significant recognition in the international art community. Some notable aspects of his art include vivid colors, intricate patterns, and an inventive blend of diverse techniques such as ink wash painting, collage, and multimedia elements.\n\nCai Xu Kun's artwork is deeply personal, and his messages often revolve around themes of identity, cultural heritage, and social commentary. His creative practice has been consist"]

# Retrieval augmented generation

In [30]:
!pip install wikipedia
!pip install wikipedia-api
from wikipediaapi import Wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=9dfd0e2814dc2c649d3229f3748625dd0b184bccffd4529f72e79c1277e6ea4e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=e289a7153e048e5684a89da2d78860d40577a88d8a9bb61f037c3f0b539047db
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [42]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [43]:
print(jh_page[:500])

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator. 
He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.

Early life
Howard was born in London, United Kingdom, and mo


In [44]:
ques = "Who is Jeremy Howard?"
len(jh_page.split())
ques_ctx = f"""Answer the question with the help of the provided context.

## Context

{jh_page}

## Question

{ques}"""

In [45]:
res = gen(mk_prompt(ques_ctx), 300)

In [46]:
print(res[0].split('### Assistant:\n')[1])

Jeremy Howard is an Australian data scientist, entrepreneur, and educator. He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning. He previously founded and led companies such as Fastmail, Optimal Decisions Group, and Enlitic. He was also President and Chief Scientist of Kaggle. Throughout his career, Howard has contributed to several open-source projects and is an advocate for the adoption of machine learning in various fields.

Also, he has made significant contributions in the area of deep learning, especially with the development of the ULMFiT algorithm for language models like the GPT family. Not only is he an educator but also an investor and frequent guest expert on Australia's most popular TV morning news program Sunrise. In addition, Howard served as an honorary professor at the School of Information Technology and Electrical Engineering at the University of Queensland. 

His interests inclu

In [47]:
from sentence_transformers import SentenceTransformer

In [48]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [49]:
jh = jh_page.split('\n\n')[0]
print(jh)

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator. 
He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.


In [50]:
tb_page = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [51]:
tb = tb_page.split('\n\n')[0]
print(tb[:380])

Sir Anthony Charles Lynton Blair  (born 6 May 1953) is a British politician who served as Prime Minister of the United Kingdom from 1997 to 2007 and Leader of the Labour Party from 1994 to 2007. He was Leader of the Opposition from 1994 to 1997 and held various shadow cabinet posts from 1987 to 1994. Blair was Member of Parliament (MP) for Sedgefield from 1983 to 2007, and was 


In [52]:
q_emb,jh_emb,tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [53]:
tb_emb.shape

torch.Size([384])

In [54]:
import torch.nn.functional as F

In [55]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

tensor(0.7991, device='cuda:0')

In [56]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

tensor(0.5328, device='cuda:0')

# Fine tuning

In [57]:
import datasets

In [58]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

README.md:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

know_sql_train-v2.json:   0%|          | 0.00/21.7M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [59]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78562
    })
})

In [61]:
trn = ds['train']
trn[6]

{'question': 'What are the maximum and minimum number of cows across all farms.',
 'context': 'CREATE TABLE farm (Cows INTEGER)',
 'answer': 'SELECT MAX(Cows), MIN(Cows) FROM farm'}

In [62]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

{'question': 'Get the count of competition hosts by theme.',
 'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'"}

In [63]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [64]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])
print(sql_prompt(tst))

SYSTEM: Use the following contextual information to concisely answer the question.

USER: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
===
Get the count of competition hosts by theme.
ASSISTANT:


In [65]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [66]:
ax_model = '/home/jhoward/git/ext/axolotl/qlora-out'

In [67]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [68]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',
                                             torch_dtype=torch.bfloat16, device_map=0)
model = PeftModel.from_pretrained(model, ax_model)
model = model.merge_and_unload()
model.save_pretrained('sql-model')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at '/home/jhoward/git/ext/axolotl/qlora-out'

In [69]:
toks = tokr(sql_prompt(tst), return_tensors="pt")
res = model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')
print(tokr.batch_decode(res)[0])

<s> SYSTEM: Use the following contextual information to concisely answer the question.

USER: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
===
Get the count of competition hosts by theme.
ASSISTANT: The following tables are involved in the system.

- farm_competition

- farm_theme

- farm_host

- farm_user

- farm_competition_theme

- farm_competition_host

- farm_competition_user

- farm_competition_user_theme

- farm_competition_user_host

- farm_competition_user_theme_host

ASSISTANT: The following contextual information is available.

- Theme: '123'

- Hosts: '123'

ASSISTANT: The following statements are available.

- SELECT COUNT(*) FROM farm_competition WHERE farm_competition.Theme = '123' AND farm_competition.Hosts = '123'

- SELECT farm_competition.Theme, COUNT(*) FROM farm_competition GROUP BY farm_competition.Theme

- SELECT farm_competition.Theme, farm_competition_theme.Theme AS theme_theme, COUNT(*) FROM


# llama.cpp

In [76]:
!pip install llama-cpp-python
from llama_cpp import Llama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510493 sha256=a07ee225e60c7661e75068336016674a004c7d746c58dd227c771c9912357da1
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python
